In [62]:
import numpy as np
import pandas as pd
from scipy import integrate
from scipy.stats import norm
from scipy.integrate import quad
from abc import ABC, abstractmethod
from typing import Union, Tuple, Dict


class MonteCarlo:

    def __init__(self, num_path: int, num_simulation: int, num_path_per_year: int = 252):
        """
        Initialize a MonteCarlo object
        @param num_path: number of path
        @param num_simulation: number of simulation
        @param num_path_per_year: number of path in a year, defaults to 252.
        """
        self.num_path = num_path
        self.num_simulation = num_simulation
        self.num_path_per_year = num_path_per_year

    def univariate_gbm(self, st: float, iv: float, d: float, b: float, r: float):
        """
        Generate univariate geometric brownian motion
        @param st: stock price
        @param iv: implied volatility
        @param d: continuously compounded dividend yield
        @param b: continuously compounded repo rate or borrowing cost
        @param r: continuously compounded risk-free interest rate
        @return: single asset generated timeseries
        """
        dt = 1 / self.num_path_per_year

        ts = np.full(shape=(self.num_path + 1, self.num_simulation), fill_value=st)

        z = np.random.standard_normal(size=self.num_simulation)

        drift = r - d - b

        for i in range(1, self.num_path + 1):
            ts[i] = ts[i - 1] * np.exp((drift - 0.5 * iv ** 2) * dt + iv * np.sqrt(dt) * z[i - 1])

        return ts

    def bivariate_gbm(self,
                      st_1: float, iv_1: float, d_1: float, b_1: float,
                      st_2: float, iv_2: float, d_2: float, b_2: float,
                      rho: float, r: float) -> Tuple[np.array, np.array]:
        """
        Generate bivariate geometric brownian motion
        @param st_1: asset 1 stock price
        @param iv_1: asset 1 implied volatility
        @param d_1: asset 1 continuously compounded dividend yield
        @param b_1: asset 1 continuously compounded repo rate or borrowing cost
        @param st_2: asset 2 stock price
        @param iv_2: asset 2 implied volatility
        @param d_2: asset 2 continuously compounded dividend yield
        @param b_2: asset 2 continuously compounded repo rate or borrowing cost
        @param rho: implied correlation between asset 1 and asset 2
        @param r: continuously compounded risk-free interest rate
        @return: 2 correlated assets generated timeseries
        """
        dt = 1 / self.num_path_per_year
        mu = np.array([0, 0])
        cov = np.array([[1, rho], [rho, 1]])

        ts_1 = np.full(shape=(self.num_path + 1, self.num_simulation), fill_value=st_1)
        ts_2 = np.full(shape=(self.num_path + 1, self.num_simulation), fill_value=st_2)

        z = np.random.multivariate_normal(mean=mu, cov=cov, size=(self.num_path, self.num_simulation))

        drift_1 = r - d_1 - b_1
        drift_2 = r - d_2 - b_2

        for i in range(1, self.num_path + 1):
            ts_1[i] = ts_1[i - 1] * np.exp((drift_1 - 0.5 * iv_1 ** 2) * dt + iv_1 * np.sqrt(dt) * z[i - 1, :, 0])
            ts_2[i] = ts_2[i - 1] * np.exp((drift_2 - 0.5 * iv_2 ** 2) * dt + iv_2 * np.sqrt(dt) * z[i - 1, :, 1])

        return ts_1, ts_2


class Priceable(ABC):
    def __init__(self):
        self.pv: Union[None, float] = None
        self.greeks: Dict[str, float] = dict()

    @abstractmethod
    def calculate_present_value(self, pricing_model: Union[None, str] = None) -> None:
        ...

    def calculate_greeks(self) -> None:
        pass

    def get_present_value(self) -> float:
        return self.pv

    def get_greeks(self):
        pass


class Forward(Priceable):

    def __init__(self, st: float, q: float, b: float, r: float, t: float, k: float = 0):
        super().__init__()
        self.st = st
        self.q = q
        self.b = b
        self.r = r
        self.t = t
        self.k = k
        
    def __post_init__(self):
        self.d = np.exp(-self.r * self.t)

    def calculate_present_value(self, pricing_model=None) -> None:
        self.__post_init__()
        self.fwd = self.st * np.exp((self.r - self.q - self.b) * self.t)
        d_strike = self.k * self.d
        d_fwd = self.fwd * self.d
        self.pv = d_fwd - d_strike

In [76]:
fwd = Forward(st=100, q=0, b=0, r=0.05, t=1, k=0)

In [77]:
fwd.calculate_present_value()

In [78]:
fwd.get_present_value()

100.00000000000001

In [66]:
fwd.fwd

91.39311852712282

In [ ]:
class Greeks:
    BUMP_MAPPING = {
        ''
    }